In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from loaders.loaders import *
from evaluation.split_dataset import split_on_seed_dataset
import json
import pickle

lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)

In [4]:
def filter_graph(graph):
    valid_triples = list()

    for triple in graph:
        if triple[0].startswith("?") and "uri" not in triple[0]:
            valid_triples.append(triple)
        if triple[2].startswith("?") and "uri" not in triple[2]:
            valid_triples.append(triple)
    return valid_triples

In [30]:
templates = list()
direction_types = {"bf1":[], "bf2":[], "fb1":[], "fb2":[], "f":[], "b":[], "ff1":[], "ff2":[], "bb1":[], "bb2":[],}

for i, record in enumerate(lc_quad_train):
    # print("NEXT:", i)
    # gold_graph_filtered = filter_graph(record["graph"])
    gold_graph_filtered = record["graph"]
    uri_direction = ""
    variable_direction = ""

    step1_type = ""
    step2_type = ""

    for triple in gold_graph_filtered:
        if triple[0].startswith("?") and "uri" not in triple[0] and "?uri" in triple[2]:
            # print("0", triple)
            uri_direction = "b"
            step1_type = triple[1]
        elif triple[0].startswith("?") and "uri" not in triple[0]:
            # print("1", triple)
            variable_direction = "f"
            step2_type = triple[1]
        elif triple[2].startswith("?") and "uri" not in triple[2] and "?uri" in triple[0]:
            # print("2", triple)
            uri_direction = "f"
            step1_type = triple[1]
        elif triple[2].startswith("?") and "uri" not in triple[2]:
            # print("3", triple)
            variable_direction = "b"
            step2_type = triple[1]
        elif "?uri" in triple[0]:
            # print("4", triple)
            uri_direction = "f"
            step1_type = triple[1]
        elif "?uri" in triple[2]:
            # print("5", triple)
            uri_direction = "b"
            step1_type = triple[1]
    if variable_direction == "":
        # print(gold_graph_filtered, record["graph"])
        direction_types[uri_direction].append(step1_type)
    else:
        direction_types[uri_direction+variable_direction+"1"].append(step1_type)
        direction_types[uri_direction+variable_direction+"2"].append(step2_type)

    templates.append(uri_direction+variable_direction)

In [28]:
from collections import Counter

Counter(templates)

Counter({'bf': 119, 'f': 69, 'ff': 48, 'b': 8, 'bb': 6})

In [11]:
125+77+48

250

In [11]:
for k, v in direction_types.items():
    print(k)
    print(sorted(Counter(v).items(), key=lambda x: x[1], reverse=True))

bf1
[('<http://dbpedia.org/ontology/sport>', 5), ('<http://dbpedia.org/ontology/bandMember>', 4), ('<http://dbpedia.org/ontology/team>', 4), ('<http://dbpedia.org/property/cities>', 4), ('<http://dbpedia.org/ontology/leader>', 4), ('<http://dbpedia.org/ontology/predecessor>', 3), ('<http://dbpedia.org/ontology/tenant>', 3), ('<http://dbpedia.org/ontology/birthPlace>', 3), ('<http://dbpedia.org/property/starring>', 3), ('<http://dbpedia.org/ontology/award>', 3), ('<http://dbpedia.org/property/subject>', 2), ('<http://dbpedia.org/property/region>', 2), ('<http://dbpedia.org/property/genre>', 2), ('<http://dbpedia.org/ontology/poleDriver>', 2), ('<http://dbpedia.org/ontology/manufacturer>', 2), ('<http://dbpedia.org/ontology/firstAscentPerson>', 2), ('<http://dbpedia.org/property/notableworks>', 2), ('<http://dbpedia.org/property/related>', 2), ('<http://dbpedia.org/ontology/associatedBand>', 2), ('<http://dbpedia.org/ontology/producer>', 2), ('<http://dbpedia.org/property/awards>', 2), (

In [12]:
for k, v in direction_types.items():
    print(k)
    print("<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>" in v)

bf1
False
bf2
True
fb1
False
fb2
False
1
False
